In [ ]:
import sys
from pathlib import Path

# Add backend to path
sys.path.insert(0, str(Path.cwd()))

from backend.models.market import Market, MarketPair, load_markets_from_parquet
import pandas as pd

# Polymarket Markets Explorer

This notebook demonstrates working with **Pydantic models** for Polymarket data:

## Object-Oriented Approach

- **Market objects**: Each market is a `Market` instance with validation and helper methods
- **MarketPair objects**: Pairs of related markets as `MarketPair` instances
- **Type safety**: Pydantic provides validation and type checking
- **Rich methods**: Objects have useful methods like `is_open()`, `has_valid_odds()`, etc.

## Placeholder for Database

The parquet files are temporary placeholders for a future database:
- `markets_to_dataframe()` - Convert Market objects to DataFrame
- `save_markets_to_parquet()` - Save to parquet (placeholder for DB insert)
- `load_markets_from_parquet()` - Load from parquet (placeholder for DB query)

Eventually, these will be replaced with proper database operations.

In [ ]:
# Load markets as Market objects (not just raw DataFrame)
markets = load_markets_from_parquet("data/markets.parquet")

print(f"Loaded {len(markets)} Market objects")
print(f"\nFirst market object:")
print(f"  Title: {markets[0].title}")
print(f"  ID: {markets[0].market_id[:30]}...")
print(f"  Yes odds: {markets[0].yes_odds}")
print(f"  No odds: {markets[0].no_odds}")
print(f"  Is open: {markets[0].is_open()}")
print(f"  Has valid odds: {markets[0].has_valid_odds()}")

# Can still convert to DataFrame when needed (placeholder for DB)
from backend.models.market import markets_to_dataframe
all_markets_df = markets_to_dataframe(markets)
all_markets_df.head()

In [ ]:
# Working with Market objects - filter for open markets
open_markets = [m for m in markets if m.is_open()]
closed_markets = [m for m in markets if m.closed]

print(f"Open markets: {len(open_markets)}")
print(f"Closed markets: {len(closed_markets)}")
print(f"\nSample open market:")
if open_markets:
    m = open_markets[0]
    print(f"  {m.title}")
    print(f"  URL: {m.url}")
    print(f"  End date: {m.end_date}")
    
# Can still use DataFrame for analysis if needed
all_markets_df.closed.value_counts()

In [ ]:
# Load market pairs as DataFrame (they're stored in flattened format)
# In the future, we could create a load_market_pairs_from_parquet function
# that reconstructs MarketPair objects from the flattened data
markets_pairs_df = pd.read_parquet("data/market_pairs.parquet")

print(f"Total market pairs: {len(markets_pairs_df)}")
print(f"\nFirst pair:")
print(f"  Keyword: {markets_pairs_df.iloc[0]['keyword']}")
print(f"  Market 1: {markets_pairs_df.iloc[0]['market1_title']}")
print(f"  Market 2: {markets_pairs_df.iloc[0]['market2_title']}")

markets_pairs_df.head()

In [ ]:
# Example: Create MarketPair objects from the data
# Find all unique market1 titles for Iran keyword
iran_pairs_df = markets_pairs_df[markets_pairs_df["keyword"] == "Iran"]
print(f"Iran-related pairs: {len(iran_pairs_df)}")
print(f"Unique Iran market 1 titles: {iran_pairs_df.market1_title.nunique()}")

# Example of working with objects - filter markets by keyword
def find_markets_by_keyword(markets_list, keyword):
    """Find markets that contain a keyword in the title."""
    import re
    pattern = re.compile(rf'\b{keyword}\b', re.IGNORECASE)
    return [m for m in markets_list if pattern.search(m.title)]

iran_markets = find_markets_by_keyword(markets, "Iran")
print(f"\nFound {len(iran_markets)} markets with 'Iran' in title (from full list)")

# Load Iran markets from the dedicated keyword parquet file
iran_markets_from_file = load_markets_from_parquet("data/keywords/Iran.parquet")
print(f"Loaded {len(iran_markets_from_file)} Iran markets from data/keywords/Iran.parquet")

# Compare the two sources
iran_titles_from_filter = {m.market_id for m in iran_markets}
iran_titles_from_file = {m.market_id for m in iran_markets_from_file}
only_in_filter = iran_titles_from_filter - iran_titles_from_file
only_in_file = iran_titles_from_file - iran_titles_from_filter
print(f"\nComparison:")
print(f"  Only in keyword filter: {len(only_in_filter)}")
print(f"  Only in parquet file:   {len(only_in_file)}")
print(f"  In both:                {len(iran_titles_from_filter & iran_titles_from_file)}")

# Display Iran markets from file
print("\nIran markets (from parquet file):")
for i, m in enumerate(iran_markets_from_file[:5]):
    print(f"{i+1}. {m.title}")
    print(f"   Yes: {m.yes_odds}, No: {m.no_odds}, Open: {m.is_open()}")

iran_markets_df = markets_to_dataframe(iran_markets_from_file)
iran_markets_df.head()

Iran-related pairs: 2850
Unique Iran market 1 titles: 75

Found 587 markets with 'Iran' in title (from full list)
Loaded 76 Iran markets from data/keywords/Iran.parquet

Comparison:
  Only in keyword filter: 437
  Only in parquet file:   0
  In both:                76

Iran markets (from parquet file):
1. Will Iran win the 2026 FIFA World Cup?
   Yes: 0.0025, No: 0.9975, Open: True
2. Khamenei out as Supreme Leader of Iran in 2026?
   Yes: 0.385, No: 0.615, Open: True
3. Khamenei out as Supreme Leader of Iran by June 30?
   Yes: 0.245, No: 0.755, Open: True
4. Will Iran close the Strait of Hormuz before 2027?
   Yes: 0.27, No: 0.73, Open: True
5. Will Iran withdraw from the NPT before 2027?
   Yes: 0.145, No: 0.855, Open: True


,market_id,title,description,url,yes_odds,no_odds,end_date,active,closed,volume,liquidity
0,0x84edef36bded182da6a395ac6c785dba8f3e09b6c5ad...,Will Iran win the 2026 FIFA World Cup?,This market will resolve according to the nati...,https://polymarket.com/event/will-iran-win-the...,0.0025,0.9975,2026-07-20 00:00:00+00:00,True,False,0.0,0.0
1,0xe6b7d4d8b4d2ff7dc8c9e5d6dd70b6eece210b8cf9e3...,Khamenei out as Supreme Leader of Iran in 2026?,"This market will resolve to ""Yes"" if Iran's Su...",https://polymarket.com/event/khamenei-out-as-s...,0.3850,0.6150,2026-12-31 00:00:00+00:00,True,False,0.0,0.0
2,0x2e94bb8dd09931d12e6e656fe4fe6ceb3922bc3d6eab...,Khamenei out as Supreme Leader of Iran by June...,"This market will resolve to ""Yes"" if Iran's Su...",https://polymarket.com/event/khamenei-out-as-s...,0.2450,0.7550,2026-06-30 00:00:00+00:00,True,False,0.0,0.0
3,0x5bbabd359667d5bbdb9ee690bb959c9aa4cf4b08b2ae...,Will Iran close the Strait of Hormuz before 2027?,If Iran halts or severely restricts internatio...,https://polymarket.com/event/will-iran-close-t...,0.2700,0.7300,2026-12-31 00:00:00+00:00,True,False,0.0,0.0
4,0x3e0845c4736ae232c57b347d05f6145650416dfd3adb...,Will Iran withdraw from the NPT before 2027?,"This market will resolve to ""Yes"" if the Islam...",https://polymarket.com/event/will-iran-withdra...,0.1450,0.8550,2026-12-31 00:00:00+00:00,True,False,0.0,0.0


In [56]:
# Example: Create MarketPair objects from the data
# Find all unique market1 titles for Trump keyword
trump_pairs_df = markets_pairs_df[markets_pairs_df["keyword"] == "Trump"]
print(f"Trump-related pairs: {len(trump_pairs_df)}")
print(f"Unique Trump market 1 titles: {trump_pairs_df.market1_title.nunique()}")

# Example of working with objects - filter markets by keyword
def find_markets_by_keyword(markets_list, keyword):
    """Find markets that contain a keyword in the title."""
    import re
    pattern = re.compile(rf'\b{keyword}\b', re.IGNORECASE)
    return [m for m in markets_list if pattern.search(m.title)]

trump_markets = find_markets_by_keyword(markets, "Trump")
print(f"\nFound {len(trump_markets)} markets with 'Trump' in title (from full list)")

# Load Trump markets from the dedicated keyword parquet file
trump_markets_from_file = load_markets_from_parquet("data/keywords/Trump.parquet")
print(f"Loaded {len(trump_markets_from_file)} Trump markets from data/keywords/Trump.parquet")

# Compare the two sources
trump_titles_from_filter = {m.market_id for m in trump_markets}
trump_titles_from_file = {m.market_id for m in trump_markets_from_file}
only_in_filter = trump_titles_from_filter - trump_titles_from_file
only_in_file = trump_titles_from_file - trump_titles_from_filter
print(f"\nComparison:")
print(f"  Only in keyword filter: {len(only_in_filter)}")
print(f"  Only in parquet file:   {len(only_in_file)}")
print(f"  In both:                {len(trump_titles_from_filter & trump_titles_from_file)}")

# Display Trump markets from file
print("\nTrump markets (from parquet file):")
for i, m in enumerate(trump_markets_from_file[:5]):
    print(f"{i+1}. {m.title}")
    print(f"   Yes: {m.yes_odds}, No: {m.no_odds}, Open: {m.is_open()}")

trump_markets_df = markets_to_dataframe(trump_markets_from_file)
trump_markets_df.head()

Trump-related pairs: 112101
Unique Trump market 1 titles: 473

Found 9879 markets with 'Trump' in title (from full list)
Loaded 474 Trump markets from data/keywords/Trump.parquet

Comparison:
  Only in keyword filter: 8658
  Only in parquet file:   0
  In both:                474

Trump markets (from parquet file):
1. Will Trump deport 500,000-750,000- people?
   Yes: 0.062, No: 0.938, Open: True
2. Will Trump deport 750,000-1,000,000 people?
   Yes: 0.0085, No: 0.9915, Open: True
3. Will Trump deport 750,000 or more people in 2025?
   Yes: 0.0625, No: 0.9375, Open: True
4. Will Trump deport 2,000,000 or more people?
   Yes: 0.006, No: 0.994, Open: True
5. Will Trump deport 1,250,000-1,500,000 people?
   Yes: 0.004, No: 0.996, Open: True


,market_id,title,description,url,yes_odds,no_odds,end_date,active,closed,volume,liquidity
0,0x2393ed0b0fdc450054c7b9071907eca75cf4fc36e385...,"Will Trump deport 500,000-750,000- people?","During the 2024 FY ICE removed 271,484 non cit...",https://polymarket.com/event/will-trump-deport...,0.0620,0.9380,2025-12-31 00:00:00+00:00,True,False,0.0,0.0
1,0x44f08744458b8896620cd3330bc5e1ea69df4199b02d...,"Will Trump deport 750,000-1,000,000 people?","During the 2024 FY ICE removed 271,484 non cit...",https://polymarket.com/event/will-trump-deport...,0.0085,0.9915,2025-12-31 00:00:00+00:00,True,False,0.0,0.0
2,0x22ac5f75af18fdb453497fbf7ac0606a09a6fd55b78b...,"Will Trump deport 750,000 or more people in 2025?","During the 2024 FY ICE removed 271,484 non cit...",https://polymarket.com/event/will-trump-deport...,0.0625,0.9375,2025-12-31 00:00:00+00:00,True,False,0.0,0.0
3,0xb9bbdd34304344dcc45803780a34c97d73a00133a5ec...,"Will Trump deport 2,000,000 or more people?","During the 2024 FY ICE removed 271,484 non cit...",https://polymarket.com/event/will-trump-deport...,0.0060,0.9940,2025-12-31 00:00:00+00:00,True,False,0.0,0.0
4,0x9472be65457510b7fc3477b20ba90910e99d92eac842...,"Will Trump deport 1,250,000-1,500,000 people?","During the 2024 FY ICE removed 271,484 non cit...",https://polymarket.com/event/will-trump-deport...,0.0040,0.9960,2025-12-31 00:00:00+00:00,True,False,0.0,0.0


In [59]:
iran_valid_markets_df = iran_markets_df[~iran_markets_df["no_odds"].isna() | ~iran_markets_df["yes_odds"].isna()]
iran_valid_markets_df.shape

(75, 11)

In [58]:
trump_valid_markets_df = trump_markets_df[~trump_markets_df["no_odds"].isna() | ~trump_markets_df["yes_odds"].isna()]
trump_valid_markets_df.shape

(474, 11)

In [62]:
NUMBER_OF_MARKETS_LIMIT = 100

# 1. Define the System Prompt with clear instructions on logic
SYSTEM_INSTRUCTIONS = """
You are an expert Market Logic Analyzer for an arbitrage trading bot. 
Your goal is to identify "Subset Markets" where the resolution of one market (Market A) strictly implies the resolution of another market (Market B).

**The Logical Rule (A -> B):**
If Market A resolves to "YES", then Market B **MUST** logically also resolve to "YES".
If this condition is met, we call this a "Risk-Free Logic Chain."

**Criteria for Implication:**
1. **Numerical Inclusion:** If A is "BTC > $100k" and B is "BTC > $90k", then A -> B.
2. **Categorical Inclusion:** If A is "Taylor Swift wins Grammy" and B is "Female Artist wins Grammy", then A -> B.
3. **Temporal Inclusion:** If A is "Event happens by Tuesday" and B is "Event happens by Friday", then A -> B.

**Constraint:**
- Ignore all prices and odds. Focus ONLY on the definitions/text.
- Ignore correlation. "Oil prices up" often means "Gas prices up", but it is not a *logical guarantee*. Mark these as NO.
"""

# 2. Provide Few-Shot Examples (Crucial for accuracy)
EXAMPLES = """
Here are examples of how you should analyze pairs:

---
**Example 1:**
Market A: "Bitcoin to hit $80k by Dec 31"
Market B: "Bitcoin to hit $75k by Dec 31"
**Analysis:** If BTC hits $80k, it must have passed $75k. The date is the same.
**Output:** MATCH (A -> B)

---
**Example 2:**
Market A: "Republicans win US Presidency"
Market B: "Donald Trump wins US Presidency"
**Analysis:** Trump is a Republican, but a different Republican could theoretically win. Trump winning implies Republicans win (B -> A), but Republicans winning does NOT imply Trump wins.
**Output:** NO MATCH (Direction wrong)

---
**Example 3:**
Market A: "SpaceX Starship launches in Q1"
Market B: "SpaceX Starship launches in 2024"
**Analysis:** Q1 is a subset of 2024. If it launches in Q1, it effectively launches in 2024.
**Output:** MATCH (A -> B)

---
**Example 4:**
Market A: "Ethereum hits $3000"
Market B: "Solana hits $200"
**Analysis:** These are correlated assets, but one happening does not force the other to happen by definition.
**Output:** NO MATCH
"""

# 3. Format your actual data (The Task)
# Note: For this to work, you usually need to pass pairs. 
# If you pass a big list, the LLM might hallucinate connections. 
# It is better to format the data as a list of potential pairs or ask it to scan a list.

# Let's assume you want it to scan a list and find pairs.
markets_list_text = ""
for index, row in trump_valid_markets_df[:NUMBER_OF_MARKETS_LIMIT].iterrows():
    markets_list_text += f"ID {index}: {row['title']} (Description: {row['description']})\n"

TASK_PROMPT = f"""
{SYSTEM_INSTRUCTIONS}

{EXAMPLES}

---
**YOUR TASK:**
Analyze the following list of markets. Identify ALL pairs where Market A -> Market B.
Return the output as a JSON list of pairs: {{"trigger_market_id": "A", "implied_market_id": "B", "reasoning": "..."}}

**MARKET LIST:**
{markets_list_text}
"""

# 4. Final variable ready to send
FULL_PROMPT = TASK_PROMPT

In [63]:
FULL_PROMPT

'\n\nYou are an expert Market Logic Analyzer for an arbitrage trading bot. \nYour goal is to identify "Subset Markets" where the resolution of one market (Market A) strictly implies the resolution of another market (Market B).\n\n**The Logical Rule (A -> B):**\nIf Market A resolves to "YES", then Market B **MUST** logically also resolve to "YES".\nIf this condition is met, we call this a "Risk-Free Logic Chain."\n\n**Criteria for Implication:**\n1. **Numerical Inclusion:** If A is "BTC > $100k" and B is "BTC > $90k", then A -> B.\n2. **Categorical Inclusion:** If A is "Taylor Swift wins Grammy" and B is "Female Artist wins Grammy", then A -> B.\n3. **Temporal Inclusion:** If A is "Event happens by Tuesday" and B is "Event happens by Friday", then A -> B.\n\n**Constraint:**\n- Ignore all prices and odds. Focus ONLY on the definitions/text.\n- Ignore correlation. "Oil prices up" often means "Gas prices up", but it is not a *logical guarantee*. Mark these as NO.\n\n\n\nHere are examples o

In [51]:
iran_markets_df.iloc[42].title

'Will Iran close the Strait of Hormuz by June 30?'

In [52]:
iran_markets_df.iloc[3].title

'Will Iran close the Strait of Hormuz before 2027?'

In [ ]:
iran_llm_answer = [
  {
    "trigger_market_id": "31",
    "implied_market_id": "13",
    "reasoning": "Temporal Inclusion: Khamenei being removed by Feb 28 strictly implies he was removed by March 31."
  },
  {
    "trigger_market_id": "13",
    "implied_market_id": "2",
    "reasoning": "Temporal Inclusion: Khamenei being removed by March 31 strictly implies he was removed by June 30."
  },
  {
    "trigger_market_id": "2",
    "implied_market_id": "1",
    "reasoning": "Temporal Inclusion: Khamenei being removed by June 30 strictly implies he was removed by December 31."
  },
  {
    "trigger_market_id": "43",
    "implied_market_id": "42",
    "reasoning": "Temporal Inclusion: Closing the Strait of Hormuz by March 31 strictly implies it was closed by June 30."
  },
  {
    "trigger_market_id": "42",
    "implied_market_id": "3",
    "reasoning": "Temporal Inclusion: Closing the Strait of Hormuz by June 30 strictly implies it was closed by December 31."
  },
  {
    "trigger_market_id": "38",
    "implied_market_id": "11",
    "reasoning": "Temporal Inclusion: An Israel strike by Feb 28 strictly implies an Israel strike by March 31."
  },
  {
    "trigger_market_id": "11",
    "implied_market_id": "10",
    "reasoning": "Temporal Inclusion: An Israel strike by March 31 strictly implies an Israel strike by June 30."
  },
  {
    "trigger_market_id": "58",
    "implied_market_id": "32",
    "reasoning": "Temporal Inclusion: A US strike by Feb 13 strictly implies a US strike by Feb 28."
  },
  {
    "trigger_market_id": "32",
    "implied_market_id": "17",
    "reasoning": "Temporal Inclusion: A US strike by Feb 28 strictly implies a US strike by March 31."
  },
  {
    "trigger_market_id": "17",
    "implied_market_id": "16",
    "reasoning": "Temporal Inclusion: A US strike by March 31 strictly implies a US strike by June 30."
  },
  {
    "trigger_market_id": "41",
    "implied_market_id": "34",
    "reasoning": "Temporal Inclusion: A US or Israel strike by Feb 15 strictly implies a strike by Feb 28."
  },
  {
    "trigger_market_id": "34",
    "implied_market_id": "33",
    "reasoning": "Temporal Inclusion: A US or Israel strike by Feb 28 strictly implies a strike by March 31."
  },
  {
    "trigger_market_id": "33",
    "implied_market_id": "40",
    "reasoning": "Temporal Inclusion: A US or Israel strike by March 31 strictly implies a strike by December 31."
  },
  {
    "trigger_market_id": "27",
    "implied_market_id": "26",
    "reasoning": "Temporal Inclusion: Trump invoking War Powers by Jan 31 strictly implies he did so by March 31."
  },
  {
    "trigger_market_id": "29",
    "implied_market_id": "25",
    "reasoning": "Temporal Inclusion: An official declaration of war by March 31 strictly implies a declaration by December 31."
  },
  {
    "trigger_market_id": "36",
    "implied_market_id": "37",
    "reasoning": "Temporal Inclusion: Reza Pahlavi entering Iran by Feb 28 strictly implies he entered by March 31."
  },
  {
    "trigger_market_id": "37",
    "implied_market_id": "18",
    "reasoning": "Temporal Inclusion: Reza Pahlavi entering Iran by March 31 strictly implies he entered by June 30."
  },
  {
    "trigger_market_id": "39",
    "implied_market_id": "19",
    "reasoning": "Temporal Inclusion: Khamenei leaving Iran by Feb 28 strictly implies he left by March 31."
  },
  {
    "trigger_market_id": "14",
    "implied_market_id": "6",
    "reasoning": "Temporal Inclusion: A nuclear deal by June 30 strictly implies a deal by December 31."
  },
  {
    "trigger_market_id": "8",
    "implied_market_id": "15",
    "reasoning": "Temporal Inclusion: A ceasefire broken by March 31 strictly implies it was broken by June 30."
  },
  {
    "trigger_market_id": "22",
    "implied_market_id": "21",
    "reasoning": "Temporal Inclusion: A military engagement by March 31 strictly implies one by June 30."
  },
  {
    "trigger_market_id": "38",
    "implied_market_id": "34",
    "reasoning": "Categorical Inclusion: An Israel strike is a specific subset of the 'US or Israel strike' condition."
  },
  {
    "trigger_market_id": "32",
    "implied_market_id": "34",
    "reasoning": "Categorical Inclusion: A US strike is a specific subset of the 'US or Israel strike' condition."
  },
  {
    "trigger_market_id": "11",
    "implied_market_id": "33",
    "reasoning": "Categorical Inclusion: An Israel strike is a specific subset of the 'US or Israel strike' condition."
  },
  {
    "trigger_market_id": "17",
    "implied_market_id": "33",
    "reasoning": "Categorical Inclusion: A US strike is a specific subset of the 'US or Israel strike' condition."
  },
  {
    "trigger_market_id": "11",
    "implied_market_id": "8",
    "reasoning": "Categorical Inclusion: An Israel strike on Iranian soil by March 31 is a specific trigger for the 'Israel x Iran ceasefire broken' market."
  },
  {
    "trigger_market_id": "28",
    "implied_market_id": "8",
    "reasoning": "Categorical Inclusion: An Iran strike on Israeli soil by Feb 28 is a specific trigger for the 'Israel x Iran ceasefire broken by March 31' market."
  },
  {
    "trigger_market_id": "17",
    "implied_market_id": "22",
    "reasoning": "Categorical Inclusion: The description for ID 22 explicitly states that any US kinetic strike on Iranian territory qualifies as military engagement."
  },
  {
    "trigger_market_id": "32",
    "implied_market_id": "22",
    "reasoning": "Categorical/Temporal Inclusion: A US strike in February is both a military engagement and occurs before the March 31 deadline for ID 22."
  },
  {
    "trigger_market_id": "35",
    "implied_market_id": "22",
    "reasoning": "Categorical/Temporal Inclusion: An Iranian strike on the US military (Feb) is a direct form of US-Iran military engagement (deadline March)."
  },
  {
    "trigger_market_id": "44",
    "implied_market_id": "75",
    "reasoning": "Categorical Identity: Market definitions are identical; A -> B and B -> A."
  },
  {
    "trigger_market_id": "75",
    "implied_market_id": "44",
    "reasoning": "Categorical Identity: Market definitions are identical; A -> B and B -> A."
  },
  {
    "trigger_market_id": "47",
    "implied_market_id": "32",
    "reasoning": "Numerical/Temporal Inclusion: The 'Next US strike' occurring on Feb 6 strictly implies a strike happened by Feb 28."
  },
  {
    "trigger_market_id": "72",
    "implied_market_id": "32",
    "reasoning": "Numerical/Temporal Inclusion: The 'Next US strike' occurring on Feb 13 strictly implies a strike happened by Feb 28."
  }
]

In [64]:
trump_llm_answer = [
  {
    "trigger_market_id": "1",
    "implied_market_id": "2",
    "reasoning": "Market 1 resolves to YES if deportations are between 750k and 1M. Market 2 resolves to YES if deportations are 750k or more. Since 750k-1M is a subset of '750k or more', 1 implies 2."
  },
  {
    "trigger_market_id": "6",
    "implied_market_id": "2",
    "reasoning": "Market 6 (1M-1.25M) is a numerical subset of Market 2 (750k or more)."
  },
  {
    "trigger_market_id": "4",
    "implied_market_id": "2",
    "reasoning": "Market 4 (1.25M-1.5M) is a numerical subset of Market 2 (750k or more)."
  },
  {
    "trigger_market_id": "5",
    "implied_market_id": "2",
    "reasoning": "Market 5 (1.5M-1.75M) is a numerical subset of Market 2 (750k or more)."
  },
  {
    "trigger_market_id": "9",
    "implied_market_id": "2",
    "reasoning": "Market 9 (1.75M-2M) is a numerical subset of Market 2 (750k or more)."
  },
  {
    "trigger_market_id": "3",
    "implied_market_id": "2",
    "reasoning": "Market 3 (2M or more) is a numerical subset of Market 2 (750k or more)."
  },
  {
    "trigger_market_id": "89",
    "implied_market_id": "76",
    "reasoning": "Market 89 resolves if approval hits 25% or below. Market 76 resolves if it hits 30% or below. Hitting 25% strictly implies that it has reached or passed 30%."
  },
  {
    "trigger_market_id": "89",
    "implied_market_id": "97",
    "reasoning": "Hitting 25% or below (Market 89) strictly implies that the approval rating has reached or passed 40% or below (Market 97)."
  },
  {
    "trigger_market_id": "76",
    "implied_market_id": "97",
    "reasoning": "Hitting 30% or below (Market 76) strictly implies that the approval rating has reached or passed 40% or below (Market 97)."
  },
  {
    "trigger_market_id": "87",
    "implied_market_id": "18",
    "reasoning": "Market 87 requires impeachment by June 30, 2026. Market 18 requires impeachment by December 31, 2026. Because June 30 occurs before December 31, any event satisfying the earlier deadline logically satisfies the later one."
  }
]

In [ ]:
# Converting objects back to DataFrame/Parquet (placeholder for DB)
from backend.models.market import market_pairs_to_dataframe

# Convert MarketPair objects to DataFrame
iran_pairs_df = market_pairs_to_dataframe(iran_pairs)

print(f"Converted {len(iran_pairs)} MarketPair objects to DataFrame")
print(f"\nDataFrame shape: {iran_pairs_df.shape}")
print("\nDataFrame columns:", list(iran_pairs_df.columns))

# Could save to parquet (placeholder for saving to database)
# from backend.models.market import save_market_pairs_to_parquet
# save_market_pairs_to_parquet(iran_pairs, "data/iran_pairs.parquet")

iran_pairs_df.head()

In [ ]:
# Example: Create MarketPair objects from Iran markets
from itertools import combinations

# Get open Iran markets with valid odds
iran_open_markets = [m for m in iran_markets if m.is_open() and m.has_valid_odds()]
print(f"Open Iran markets with valid odds: {len(iran_open_markets)}")

# Create some MarketPair objects
iran_pairs = []
for i, (m1, m2) in enumerate(combinations(iran_open_markets[:5], 2)):
    pair = MarketPair(
        pair_id=f"Iran_{i+1:04d}",
        keyword="Iran",
        market1=m1,
        market2=m2
    )
    iran_pairs.append(pair)

print(f"\nCreated {len(iran_pairs)} MarketPair objects")

# Show first pair
if iran_pairs:
    pair = iran_pairs[0]
    print(f"\nSample pair:")
    print(f"  Pair ID: {pair.pair_id}")
    print(f"  Keyword: {pair.keyword}")
    print(f"  Market 1: {pair.market1.title[:60]}...")
    print(f"  Market 1 odds: Yes={pair.market1.yes_odds}, No={pair.market1.no_odds}")
    print(f"  Market 2: {pair.market2.title[:60]}...")
    print(f"  Market 2 odds: Yes={pair.market2.yes_odds}, No={pair.market2.no_odds}")
    print(f"  Both open: {pair.both_markets_open()}")
    print(f"  Both have valid odds: {pair.both_have_valid_odds()}")